<a href="https://colab.research.google.com/github/VJander/Maule-Basin/blob/main/Tesis_MSc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip uninstall numpy
%pip install numpy

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Tesis
%ls

In [ ]:
%pip install Platypus-Opt
%pip install pywr

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import platypus
import pywr
from pywr.optimisation.platypus import PlatypusWrapper
from Maule_parameters import *

# Load the Model
def get_model_data():
    with open('PruebaMauleFinal_ConMedidas.json') as fh:
        data = json.load(fh)
    return data

# Define platypus algorithm, and variable storing
def platypus_main():
    wrapper = PlatypusWrapper(get_model_data())
    with platypus.ProcessPoolEvaluator() as evaluator:
        algorithm = platypus.NSGAIII(wrapper.problem, population_size=50, evaluator=evaluator, divisions_outer=6)
        algorithm.run(10000) 

    objective_names = []
    objective_directions = []
    for o in wrapper.model_objectives:
        direction = 'MIN' if o.is_objective == 'minimise' else 'MAX'        
        n = 'MO_{}_{}'.format(direction, o.name.replace('_', ' '))
        objective_names.append(n)
        objective_directions.append(1 if o.is_objective == 'minimise' else -1)
    objective_directions = np.array(objective_directions)
    objectives = pd.DataFrame([s.objectives[:]*objective_directions for s in algorithm.result], columns=objective_names)
    variables = pd.DataFrame([s.variables[:] for s in algorithm.result],
                    columns=['VAR_{}'.format(p.name.replace('_', ' ')) for p in wrapper.model_variables])
    recorders = pd.DataFrame([s.objectives[:] for s in algorithm.result],
                    columns=['REC_{}'.format(p.name.replace('_', ' ')) for p in wrapper.model_objectives])

    df = pd.concat([objectives, variables, recorders], axis=1)
    df.index.name = 'ID'
    df.to_csv('Maule Results_S14.csv')  
    
    # from pandas.plotting import scatter_matrix
    # scatter_matrix(objectives)
    # plt.savefig('Two Reservoir Objectives.pdf', format='pdf')
    # plt.show()

# Use platypus created function
if __name__ == '__main__':
    platypus_main()

# Print to verify it runs
print("It worked!!")

¡¡Corrio todo carajo!!
